In [1]:
import os
import random
import math
import pygame
from os import listdir
from os.path import isfile, join
pygame.init()

pygame.display.set_caption("Platformer")


WIDTH, HEIGHT = 800, 600
FPS = 60
PLAYER_VEL = 5
SCROLL = False

window = pygame.display.set_mode((WIDTH, HEIGHT))

def flip(sprites):
    return [pygame.transform.flip(sprite, True, False) for sprite in sprites]

def load_sprite_sheets(dir1, dir2, width, height, direction = False):
    path = join("assets", dir1, dir2)
    images = [f for f in listdir(path) if isfile(join(path, f))]

    all_sprites = {}

    for image in images:
        sprite_sheet = pygame.image.load(join(path, image)).convert_alpha()

        sprites = []
        for i in range(sprite_sheet.get_width() // width):
            surface = pygame.Surface((width, height), pygame.SRCALPHA, 32)
            rect = pygame.Rect(i * width, 0, width, height)
            surface.blit(sprite_sheet, (0, 0), rect)
            sprites.append(pygame.transform.scale2x(surface))

        if direction:
            all_sprites[image.replace(".png", "") + "_right"] = sprites
            all_sprites[image.replace(".png", "") + "_left"] = flip(sprites)

        else:
            all_sprites[image.replace(".png", "")] = sprites

    return all_sprites   


def get_block(size):
    path = join("assets", "Terrain", "Terrain.png")
    image = pygame.image.load(path).convert_alpha()
    surface = pygame.Surface((size, size), pygame.SRCALPHA, 32)
    rect = pygame.Rect(96, 0, size, size)
    surface.blit(image, (0, 0), rect)
    return pygame.transform.scale2x(surface)

class Player(pygame.sprite.Sprite):
    COLOR = (255, 0, 0)
    GRAVITY = 1
    SPRITES = load_sprite_sheets("MainCharacters", "MaskDude", 32, 32, True)
    ANIMATION_DELAY = 5
    
    def __init__(self, x, y, width, height):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.x_vel = 0
        self.y_vel = 0
        self.mask = None
        self.direction = "left"
        self.animation_count = 0
        self.fall_count = 0
        self.jump_count = 0

    def jump(self):
        self.y_vel = -self.GRAVITY * 8
        self.animation_count = 0
        self.jump_count += 1
        if self.jump_count == 1:
            self.fall_count = 0

    
    def move(self, dx, dy):
        self.rect.x += dx
        self.rect.y += dy

    def move_left(self, vel):
        self.x_vel = -vel
        if self.direction != "left":
            self.direction = "left"
            self.animation_count = 0

    def move_right(self, vel):
        self.x_vel = vel
        if self.direction != "right":
            self.direction = "right"
            self.animation_count = 0

    def loop(self, fps):
        self.y_vel += min(1, (self.fall_count / fps) * self.GRAVITY)
        self.move(self.x_vel, self.y_vel)

        self.fall_count += 1
        self.update_sprite()

    def update_sprite(self):
        sprite_sheet = "idle"
        if self.x_vel != 0:
            sprite_sheet = "run"

        sprite_sheet_name = sprite_sheet + "_" + self.direction
        sprites = self.SPRITES[sprite_sheet_name]
        sprite_index = (self.animation_count // self.ANIMATION_DELAY) % len(sprites)
        self.sprite = sprites[sprite_index]
        self.animation_count += 1
        self.update()

    def update(self):
        self.rect = self.sprite.get_rect(topleft=(self.rect.x, self.rect.y))
        self.mask = pygame.mask.from_surface(self.sprite)
        
    def landed(self):
        self.fall_count = 0
        self.y_vel = 0
        self.jump_count = 0

    def hit_head(self):
        self.count = 0
        self.y_vel *= -1
    
    def draw(self, win):
        self.sprite = self.SPRITES["idle_" + self.direction][0]
        win.blit(self.sprite, (self.rect.x, self.rect.y))


class FallingBlock(pygame.sprite.Sprite):
    COLOR = (0, 255, 0)
    GRAVITY = 0.5
    SPRITES = load_sprite_sheets("Terrain", "Terrain", 32, 32, True)
    ANIMATION_DELAY = 5
    
    def __init__(self, x, y, width, height):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.x_vel = 0
        self.y_vel = 0
        self.mask = None
        self.direction = "left"
        self.animation_count = 0
        self.fall_count = 0
        self.jump_count = 0

    def jump(self):
        self.y_vel = -self.GRAVITY * 8
        self.animation_count = 0
        self.jump_count += 1
        if self.jump_count == 1:
            self.fall_count = 0

    
    def move(self, dx, dy):
        self.rect.x += dx
        self.rect.y += dy

    def move_left(self, vel):
        self.x_vel = -vel
        if self.direction != "left":
            self.direction = "left"
            self.animation_count = 0

    def move_right(self, vel):
        self.x_vel = vel
        if self.direction != "right":
            self.direction = "right"
            self.animation_count = 0

    def loop(self, fps):
        #self.y_vel += min(0.2, (self.fall_count / fps) * self.GRAVITY)
        self.y_vel += 0.2
        self.move(self.x_vel, self.y_vel)

        self.fall_count += 1
        self.update_sprite()

    def update_sprite(self):
        sprite_sheet = "idle"
        if self.x_vel != 0:
            sprite_sheet = "run"

        sprite_sheet_name = sprite_sheet + "_" + self.direction
        sprites = self.SPRITES[sprite_sheet_name]
        sprite_index = (self.animation_count // self.ANIMATION_DELAY) % len(sprites)
        self.sprite = sprites[sprite_index]
        self.animation_count += 1
        self.update()

    def update(self):
        self.rect = self.sprite.get_rect(topleft=(self.rect.x, self.rect.y))
        self.mask = pygame.mask.from_surface(self.sprite)
        
    def landed(self):
        self.fall_count = 0
        self.y_vel = 0
        self.jump_count = 0

    def hit_head(self):
        self.count = 0
        self.y_vel *= -1
    
    def draw(self, win):
        self.sprite = self.SPRITES["idle_" + self.direction][0]
        win.blit(self.sprite, (self.rect.x, self.rect.y))


class Object(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height, name = None):
        super().__init__()
        self.rect = pygame.Rect(x, y, width, height)
        self.image = pygame.Surface((width, height), pygame.SRCALPHA)
        self.width = width
        self.height = height
        self.name = name

    def draw(self, win):
        win.blit(self.image, (self.rect.x, self.rect.y))

class Block(Object):
    def __init__(self, x, y, size):
        super().__init__(x, y, size, size)
        block = get_block(size)
        self.image.blit(block, (0, 0))
        self.mask = pygame.mask.from_surface(self.image)


def get_background(name): 
    image = pygame.image.load(join("assets", "Background", name))
    _, _, width, height = image.get_rect()
    tiles = []

    for i in range(WIDTH // width + 1):
        for j in range(HEIGHT // height + 1):
            pos = (i * width, j * height)
            tiles.append(pos)

    return tiles, image
    

def draw(window, background, bg_image, player, objects, objects2):
    for tile in background: 
        window.blit(bg_image, tile)

    for obj in objects:
        obj.draw(window)

    for obj in objects2:
        obj.draw(window)

    player.draw(window)

    pygame.display.update()


def handle_vertical_collision(player, objects, dy):
    collided_objects = []
    for obj in objects: 
        if pygame.sprite.collide_mask(player, obj):
            if dy > 0:
                player.rect.bottom = obj.rect.top
                player.landed()
            elif dy < 0:
                player.rect.top = obj.rect.bottom
                player.hit_head()

        collided_objects.append(obj)

    return collided_objects
    

def collide(player, objects, dx) :
    player.move(dx, 0)
    player.update()
    collided_object = None 
    for obj in objects :
        if pygame.sprite.collide_mask(player, obj) : 
            collided_object = obj
            break 

    player.move(-dx, 0)
    player.update()
    return collided_object
                
def handle_move(player, objects):
    keys = pygame.key.get_pressed()

    player.x_vel = 0
    collide_left = collide(player, objects, -PLAYER_VEL * 2)
    collide_right = collide(player, objects, PLAYER_VEL * 2)
    
    if keys[pygame.K_a] and not collide_left:
        player.move_left(PLAYER_VEL)
    if keys[pygame.K_d] and not collide_right:
        player.move_right(PLAYER_VEL)

    handle_vertical_collision(player, objects, player.y_vel)

def handle_move_forFallingBlock(player, objects):
    keys = pygame.key.get_pressed()

    player[0].x_vel = 0
    collide_left = collide(player[0], objects, -PLAYER_VEL * 2)
    collide_right = collide(player[0], objects, PLAYER_VEL * 2)
    
    if keys[pygame.K_LEFT] and not collide_left:
        player[0].move_left(PLAYER_VEL)
    if keys[pygame.K_RIGHT] and not collide_right:
        player[0].move_right(PLAYER_VEL)

    handle_vertical_collision_forFallingBlock(player, objects, player[0].y_vel)
    
def handle_vertical_collision_forFallingBlock(player, objects, dy):
    collided_objects = []
    for obj in objects: 
        if pygame.sprite.collide_mask(player[0], obj):
            player[0].rect.bottom = obj.rect.top
            player[0].landed()
            player[1] = True

        collided_objects.append(obj)

    return collided_objects

    
def scrolling(blocks, player, floor):
    if len(blocks) > 3:
        for i in range(len(blocks)):
            blocks[i].rect.y += 1
            for i in range(len(floor)):
                floor[i].rect.y += 0.5

    for j in range(len(blocks)):
        if blocks[j].rect.y >= HEIGHT:
            blocks[j].remove
    
    if player.rect.y >= HEIGHT:
        pygame.quit()


def main(window): 
    clock = pygame.time.Clock()
    background, bg_image = get_background("Blue.png")

    block_size = 96

    block_spawn_timer = 0
    block_spawn_interval = 100
    
    player = Player(100, 100, 50, 50)
    fall = FallingBlock(50, 200, 50, 50)
    floor = [Block(i * block_size, HEIGHT - block_size, block_size) 
             for i in range(-WIDTH // block_size, (WIDTH * 2) // block_size)]
    objects =[*floor]
    objects2 = []
  
    run = True
    while run:
        clock.tick(FPS)

        block_spawn_timer += 1 
        if block_spawn_timer >= block_spawn_interval :
            new_block_size = block_size
            new_block_x = random.randint(0, WIDTH - new_block_size)
            # new_block = Block2(new_block_x, 0, new_block_size)
            new_fall = FallingBlock(new_block_x, 0, 50, 50)
            # counter = 0
            # if len(objects2) != 0:
                # counter = objects2[len(objects2)-1][1] + 1
            objects2.append([new_fall, False])
            block_spawn_timer = 0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and player.jump_count < 2:
                    player.jump()

        if len(objects2) != 0 and objects2[len(objects2)-1][1] == False:
            curr_block = objects2[len(objects2)-1]
            curr_block[0].loop(FPS)
            handle_move_forFallingBlock(curr_block, objects)
            objects4 = []
            for i in range(0, len(objects2)-1):
                objects4.append(objects2[i][0])
            handle_move_forFallingBlock(curr_block, objects4)
        

        player.loop(FPS)
        handle_move(player, objects)
        objects3 = []
        for i in objects2:
            objects3.append(i[0])
        handle_move(player, objects3)
        draw(window, background, bg_image, player, objects, objects3)
        print(floor)
        scrolling(objects3, player, floor)
        
    pygame.quit()
    quit()

if __name__ == "__main__":
    main(window)

pygame 2.5.2 (SDL 2.28.3, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
[<Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>]
[<Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in 0 groups)>, <Block Sprite(in

: 